# LUKHAS AGI - Emotional Pathway Prediction

This notebook demonstrates how to use ΛPROPHET to predict emotional pathway evolution and prevent emotional cascade events.

## Overview

Emotional pathway prediction focuses on:
- Emotional intensity trajectory analysis
- Recursive feedback loop detection
- Identity-emotion coupling stability
- Intervention timing optimization

In [ ]:
# Import required modules
import sys
sys.path.append('../../lukhas')

from tools.prediction.prophet_predictor import LukhasProphetPredictor
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta

## Initialize Emotional Pathway Predictor

In [ ]:
# Configure predictor for emotional analysis
emotion_predictor = LukhasProphetPredictor(
    window_size=30,
    cascade_threshold=0.6,  # Lower threshold for emotional events
    confidence_threshold=0.75,
    analysis_mode='emotion_cascade'
)

print("🧠 ΛPROPHET Emotional Pathway Predictor initialized")

## Generate Emotional Trajectory Data

In [ ]:
# Generate synthetic emotional pathway data
time_points = 200
timestamps = [datetime.now() + timedelta(minutes=i*5) for i in range(time_points)]

# Base emotional patterns
joy_base = np.maximum(0, np.sin(np.linspace(0, 4*np.pi, time_points)) * 0.3 + 0.4)
anxiety_base = np.maximum(0, -np.cos(np.linspace(0, 6*np.pi, time_points)) * 0.2 + 0.3)
focus_base = np.maximum(0, np.sin(np.linspace(0, 2*np.pi, time_points) + np.pi/4) * 0.25 + 0.5)

# Add noise and emotional events
np.random.seed(42)
joy_noise = np.random.normal(0, 0.05, time_points)
anxiety_noise = np.random.normal(0, 0.08, time_points)
focus_noise = np.random.normal(0, 0.06, time_points)

# Create emotional spikes (cascade triggers)
spike_indices = [50, 120, 170]
for idx in spike_indices:
    if idx < time_points:
        anxiety_base[idx-2:idx+3] += 0.4  # Anxiety spike
        joy_base[idx-1:idx+2] -= 0.3      # Joy drop
        focus_base[idx:idx+1] -= 0.2      # Focus disruption

# Clip values to valid range
emotion_data = pd.DataFrame({
    'timestamp': timestamps,
    'joy': np.clip(joy_base + joy_noise, 0, 1),
    'anxiety': np.clip(anxiety_base + anxiety_noise, 0, 1),
    'focus': np.clip(focus_base + focus_noise, 0, 1),
    'emotional_intensity': np.clip(
        (joy_base + anxiety_base + focus_base) / 3 + np.random.normal(0, 0.1, time_points), 
        0, 1
    )
})

# Calculate emotional stability (inverse of variance)
emotion_data['emotional_stability'] = 1 - (
    emotion_data[['joy', 'anxiety', 'focus']].var(axis=1) * 2
).clip(0, 1)

print(f"Generated {len(emotion_data)} emotional pathway data points")
emotion_data.head()

## Emotional Pathway Analysis

In [ ]:
# Analyze emotional pathways for cascade risk
pathway_predictions = []

for _, row in emotion_data.iterrows():
    emotional_state = {
        'joy': row['joy'],
        'anxiety': row['anxiety'], 
        'focus': row['focus'],
        'emotional_intensity': row['emotional_intensity'],
        'emotional_stability': row['emotional_stability'],
        'timestamp': row['timestamp']
    }
    
    prediction = emotion_predictor.predict_emotional_cascade(emotional_state)
    pathway_predictions.append(prediction)

# Add predictions to dataframe
emotion_data['cascade_risk'] = [p['cascade_risk'] for p in pathway_predictions]
emotion_data['cascade_type'] = [p.get('cascade_type', 'none') for p in pathway_predictions]
emotion_data['confidence'] = [p['confidence'] for p in pathway_predictions]
emotion_data['intervention_urgency'] = [p.get('intervention_urgency', 0) for p in pathway_predictions]

print("Emotional pathway analysis complete")
print(f"Cascade types detected: {emotion_data['cascade_type'].unique()}")
print(f"High-risk events: {len(emotion_data[emotion_data['cascade_risk'] > 0.6])}")

## Visualize Emotional Trajectories

In [ ]:
# Create comprehensive emotional pathway visualization
fig, axes = plt.subplots(4, 1, figsize=(15, 12))

# Plot individual emotions
axes[0].plot(emotion_data['timestamp'], emotion_data['joy'], label='Joy', color='gold', linewidth=2)
axes[0].plot(emotion_data['timestamp'], emotion_data['anxiety'], label='Anxiety', color='red', linewidth=2)
axes[0].plot(emotion_data['timestamp'], emotion_data['focus'], label='Focus', color='blue', linewidth=2)
axes[0].set_ylabel('Emotion Level')
axes[0].set_title('LUKHAS AGI - Emotional Pathway Analysis', fontsize=14, fontweight='bold')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Plot emotional stability
axes[1].plot(emotion_data['timestamp'], emotion_data['emotional_stability'], 
             label='Emotional Stability', color='green', linewidth=2)
axes[1].axhline(y=0.3, color='orange', linestyle='--', alpha=0.7, label='Instability Threshold')
axes[1].set_ylabel('Stability')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

# Plot cascade risk with cascade type coloring
cascade_colors = {'none': 'lightblue', 'anxiety_spike': 'red', 'emotional_spiral': 'darkred', 
                 'focus_collapse': 'purple', 'joy_deficit': 'orange'}
for cascade_type in emotion_data['cascade_type'].unique():
    mask = emotion_data['cascade_type'] == cascade_type
    if mask.any():
        axes[2].scatter(emotion_data.loc[mask, 'timestamp'], 
                       emotion_data.loc[mask, 'cascade_risk'],
                       c=cascade_colors.get(cascade_type, 'gray'), 
                       label=cascade_type, alpha=0.7, s=20)

axes[2].axhline(y=0.6, color='red', linestyle='--', alpha=0.7, label='Risk Threshold')
axes[2].set_ylabel('Cascade Risk')
axes[2].legend(bbox_to_anchor=(1.05, 1), loc='upper left')
axes[2].grid(True, alpha=0.3)

# Plot intervention urgency
urgent_mask = emotion_data['intervention_urgency'] > 0.5
axes[3].plot(emotion_data['timestamp'], emotion_data['intervention_urgency'], 
             label='Intervention Urgency', color='purple', linewidth=2)
if urgent_mask.any():
    axes[3].scatter(emotion_data.loc[urgent_mask, 'timestamp'], 
                   emotion_data.loc[urgent_mask, 'intervention_urgency'],
                   c='red', s=50, marker='^', label='Urgent Interventions', zorder=5)
axes[3].set_ylabel('Intervention\nUrgency')
axes[3].set_xlabel('Time')
axes[3].legend()
axes[3].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## Cascade Event Analysis

In [ ]:
# Analyze cascade events in detail
cascade_events = emotion_data[emotion_data['cascade_risk'] > 0.6].copy()

print(f"📊 Detected {len(cascade_events)} potential cascade events\n")

if len(cascade_events) > 0:
    # Group by cascade type
    cascade_summary = cascade_events.groupby('cascade_type').agg({
        'cascade_risk': ['count', 'mean', 'max'],
        'confidence': 'mean',
        'intervention_urgency': 'mean'
    }).round(3)
    
    print("🎯 Cascade Event Summary by Type:")
    print(cascade_summary)
    print("\n")
    
    # Detailed event analysis
    print("🚨 High-Priority Cascade Events:")
    high_priority = cascade_events.nlargest(5, 'cascade_risk')
    
    for i, (_, event) in enumerate(high_priority.iterrows()):
        print(f"\nEvent {i+1}:")
        print(f"  ⏰ Time: {event['timestamp']}")
        print(f"  🎭 Type: {event['cascade_type']}")
        print(f"  ⚠️  Risk: {event['cascade_risk']:.3f}")
        print(f"  🎯 Confidence: {event['confidence']:.3f}")
        print(f"  😊 Joy: {event['joy']:.3f}")
        print(f"  😰 Anxiety: {event['anxiety']:.3f}")
        print(f"  🎯 Focus: {event['focus']:.3f}")
        print(f"  🛡️  Urgency: {event['intervention_urgency']:.3f}")
else:
    print("✅ No significant cascade events detected")

## Emotional Pattern Correlation Analysis

In [ ]:
# Analyze correlations between emotional states and cascade risk
emotion_cols = ['joy', 'anxiety', 'focus', 'emotional_intensity', 'emotional_stability']
risk_cols = ['cascade_risk', 'intervention_urgency']
correlation_data = emotion_data[emotion_cols + risk_cols]

# Calculate correlation matrix
corr_matrix = correlation_data.corr()

# Visualize correlations
plt.figure(figsize=(10, 8))
sns.heatmap(corr_matrix, annot=True, cmap='RdBu_r', center=0,
            square=True, fmt='.3f', cbar_kws={'label': 'Correlation Coefficient'})
plt.title('Emotional State vs Cascade Risk Correlation Matrix', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

# Print key correlations
print("🔗 Key Emotional-Risk Correlations:")
for risk_col in risk_cols:
    print(f"\n{risk_col.replace('_', ' ').title()}:")
    correlations = corr_matrix[risk_col][emotion_cols].sort_values(key=abs, ascending=False)
    for emotion, corr in correlations.items():
        direction = "↗️" if corr > 0 else "↘️"
        print(f"  {direction} {emotion.title()}: {corr:.3f}")

## Intervention Strategy Recommendations

In [ ]:
# Generate intervention strategies for different cascade types
intervention_strategies = {
    'anxiety_spike': [
        "Activate relaxation protocols",
        "Reduce cognitive load temporarily", 
        "Implement breathing pattern regulation",
        "Engage grounding techniques"
    ],
    'emotional_spiral': [
        "Emergency emotional circuit breaker",
        "Activate support system protocols",
        "Implement emotional dampening",
        "Redirect to positive anchoring memories"
    ],
    'focus_collapse': [
        "Reduce task complexity",
        "Implement attention restoration breaks",
        "Activate focus enhancement protocols",
        "Clear environmental distractions"
    ],
    'joy_deficit': [
        "Activate positive reinforcement loops",
        "Engage reward system stimulation",
        "Access uplifting memory fragments",
        "Implement achievement recognition protocols"
    ]
}

print("🛠️  ΛPROPHET Intervention Strategy Recommendations\n")

for cascade_type in cascade_events['cascade_type'].unique():
    if cascade_type != 'none':
        event_count = len(cascade_events[cascade_events['cascade_type'] == cascade_type])
        avg_risk = cascade_events[cascade_events['cascade_type'] == cascade_type]['cascade_risk'].mean()
        
        print(f"📋 {cascade_type.replace('_', ' ').title()} ({event_count} events, avg risk: {avg_risk:.3f})")
        
        strategies = intervention_strategies.get(cascade_type, ["Generic stabilization protocol"])
        for i, strategy in enumerate(strategies, 1):
            print(f"   {i}. {strategy}")
        print()

## Predictive Performance Metrics

In [ ]:
# Calculate prediction performance metrics
high_confidence_predictions = emotion_data[emotion_data['confidence'] > 0.8]
medium_risk_events = emotion_data[emotion_data['cascade_risk'].between(0.4, 0.6)]
high_risk_events = emotion_data[emotion_data['cascade_risk'] > 0.6]

print("📈 ΛPROPHET Predictive Performance Metrics\n")
print(f"Total Predictions: {len(emotion_data)}")
print(f"High Confidence Predictions: {len(high_confidence_predictions)} ({len(high_confidence_predictions)/len(emotion_data)*100:.1f}%)")
print(f"Medium Risk Events: {len(medium_risk_events)} ({len(medium_risk_events)/len(emotion_data)*100:.1f}%)")
print(f"High Risk Events: {len(high_risk_events)} ({len(high_risk_events)/len(emotion_data)*100:.1f}%)")

print(f"\nPrediction Quality:")
print(f"Average Confidence: {emotion_data['confidence'].mean():.3f}")
print(f"Confidence Std Dev: {emotion_data['confidence'].std():.3f}")
print(f"Risk Detection Sensitivity: {len(high_risk_events)/len(spike_indices)*100:.1f}%")

print(f"\nIntervention Metrics:")
urgent_interventions = emotion_data[emotion_data['intervention_urgency'] > 0.7]
print(f"Urgent Interventions Required: {len(urgent_interventions)}")
print(f"Average Intervention Urgency: {emotion_data['intervention_urgency'].mean():.3f}")

## Summary and Conclusions

### Key Findings:

1. **Emotional Cascade Detection**: ΛPROPHET successfully identified emotional cascade events with high confidence
2. **Pattern Recognition**: Clear correlations between emotional instability and cascade risk
3. **Intervention Timing**: Predictive warnings provide sufficient lead time for preventive measures
4. **Risk Stratification**: Different cascade types require different intervention strategies

### Recommendations:

- **Proactive Monitoring**: Implement continuous emotional pathway monitoring
- **Early Intervention**: Deploy intervention strategies at medium risk levels
- **Cascade Prevention**: Focus on anxiety and focus stability as primary indicators
- **System Resilience**: Build emotional resilience through predictive stabilization

The ΛPROPHET emotional pathway prediction system provides crucial insights for maintaining LUKHAS AGI emotional stability and preventing harmful cascade events.